# Aula 5 - Desafío: ¡Vamos a practicar!



---

Crear una base de datos local de clientes para una institución financiera. Disponemos del archivo CSV con los datos de los clientes.

1.   Crear la base de datos local con la biblioteca SQLAlchemy.

In [1]:
#Crear Banco de Datos Local
import pandas as pd
import sqlalchemy

In [2]:
# Biblioteca sqlalquemy
# create_engine : crea el motor de datos que va a correr en nuuestro banco de datos
# MetaData : guarda la información de las tablas que vamos a tener en el banco de datos, atributos, tipos de datos, diccionario de todas las tablas y sus columnas
# Table
# inspect : inspeccionar bancos de datos, que tablas tengo, que atributos tengo dentro de la tabla, metodo investigador que me dice que hay en algun lugar
# text : convertir las querys en un tipo de datos de texto que se entiende el as consultas
from sqlalchemy import create_engine, MetaData, Table, inspect, text
#Importar bibliotec SQLAlchemyError de sqlalquemy para saber que tipo de error es en caso de suceder
from sqlalchemy.exc import SQLAlchemyError

In [3]:
# 1 . Crear Banco de Datos Local, el motor que va a soportar mis tablas. Banco de datos que ya viene instalado en Google Colab 'SQLite', no se necesita instalar otro paquete externo ni instalar nada mas. Existen otros MySql, Oracle, Sql Server etc.
#selecciono el motor y digo donde quiero que se ejecute en este caso en la memoria de google colab
#almacenarla en algun lugar para relacionarlo un parametro engine en este caso
engine = create_engine('sqlite:///:memory:')

2.  Escribir los datos del archivo CSV en esta base de datos local.

In [4]:
archivo = 'clientes_banco.csv'

In [5]:
df = pd.read_csv(archivo)

In [6]:
#crear tabla clientes con los datos del Data Frame datos
df.to_sql('clientes',engine, index=False)

438463

In [7]:
df.head()

,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda
0,5008804,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
1,5008805,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
2,5008806,58,Nivel intermedio,Casado,2,Empleado,Seguridad,3,112500.0,1,Casa/Departamento propio
3,5008808,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio
4,5008809,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio


In [14]:
# Validar que creo la tabla clientes, parametro que reciba la ejecucion del inspector
inspector = inspect(engine)

In [15]:
print(inspector.get_table_names())
# Aqui observo que ya hay una tabla con el nombre clientes

['clientes']


3. Realizar tres actualizaciones en la base de datos:

* Actualizar el registro del cliente ID 6840104 cuyo rendimiento anual cambió a 300000.

In [10]:
#Filtar solo clientes con categoria de renta = empleado
query = 'SELECT Rendimiento_anual, * FROM clientes WHERE ID_Cliente = 6840104'

In [11]:
#Ejecutar query
pd.read_sql(sql = text(query), con = engine.connect())

,Rendimiento_anual,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda
0,135000.0,6840104,62,Nivel intermedio,Divorciado,1,Pensionista,Otro,0,135000.0,0,Casa/Departamento propio


In [20]:
#Actualizando registros
query = "UPDATE clientes SET Rendimiento_anual = 300000 WHERE ID_Cliente = 6840104"
try:
  #ejecuta setencia UPDATE no devuelve registros
  r_set = engine.connect().execute(text(query))
except SQLAlchemyError as e:
   print(e)
else:
   print("#Registros actualizados: ", r_set.rowcount)

#Registros actualizados:  1


In [21]:
pd.read_sql(sql = text('SELECT * FROM clientes WHERE ID_Cliente = 6840104'), con = engine.connect())

,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda
0,6840104,62,Nivel intermedio,Divorciado,1,Pensionista,Otro,0,300000.0,0,Casa/Departamento propio


* Eliminar el registro de cliente ID 5008809, ya que esta persona ya no tiene cuenta en la institución financiera.

In [22]:
pd.read_sql(sql = text('SELECT * FROM clientes WHERE ID_Cliente = 5008809'), con = engine.connect())

,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda
0,5008809,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio


In [23]:
#Borrar en SQL
query = "DELETE FROM clientes WHERE ID_Cliente = 5008809"
try:
  #ejecuta setencia DELETE no devuelve registros
  r_set = engine.connect().execute(text(query))
except SQLAlchemyError as e:
   print(e)
else:
   print("#Registros borrados: ", r_set.rowcount)

#Registros borrados:  1


In [24]:
pd.read_sql(sql = text('SELECT * FROM clientes WHERE ID_Cliente = 5008809'), con = engine.connect())

,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda


* Crear un nuevo registro de cliente siguiendo las especificaciones a continuación:
  - ID_cliente: 6850985
  - Edad: 33
  - Grado_estudio: Doctorado
  - Estado_civil: Soltero
  - Tamaño_familia: 1
  - Categoria_de_renta: Empleado
  - Ocupacion: TI
  - Años_empleado: 2
  - Rendimiento_anual: 290000
  - Tiene_carro: 0
  - Vivienda: Casa/Departamento propio

In [26]:
# Listar nombre de columnas separados en comillas y con , esto ahorra mucho tiempo que hacia antes en excel cuando necesitaba hacer estos INSERT INTO
print (list(df.columns))

['ID_Cliente', 'Edad', 'Grado_estudio', 'Estado_civil', 'Tamaño_familia', 'Categoria_de_renta', 'Ocupacion', 'Años_empleado', 'Rendimiento_anual', 'Tiene_carro', 'Vivienda']


In [27]:
# otra forma de lista columnas
print (df.columns.tolist())

['ID_Cliente', 'Edad', 'Grado_estudio', 'Estado_civil', 'Tamaño_familia', 'Categoria_de_renta', 'Ocupacion', 'Años_empleado', 'Rendimiento_anual', 'Tiene_carro', 'Vivienda']


In [29]:
#validar tipos de datos para saber si estoy enviando un tipo de dato correcto al campo para el INSERT
df.dtypes

,0
ID_Cliente,int64
Edad,int64
Grado_estudio,object
Estado_civil,object
Tamaño_familia,int64
Categoria_de_renta,object
Ocupacion,object
Años_empleado,int64
Rendimiento_anual,float64
Tiene_carro,int64


In [43]:
#query de inseción
query = "INSERT INTO clientes (ID_Cliente, Edad, Grado_estudio, Estado_civil, Tamaño_familia, Categoria_de_renta, Ocupacion, Años_empleado, Rendimiento_anual, Tiene_carro, Vivienda) VALUES (6850985,33,'Doctorado','Soltero',1,'Empleado','TI',2,290000,0,'Casa/Departamento')"
try:
    #ejecutar INSERT
    r_set = engine.connect().execute(text(query))
except SQLAlchemyError as e:
    print(e)
else:
    print("#Registros insertados: ", r_set.rowcount)

#Registros insertados:  1


In [46]:
query = 'SELECT * FROM clientes WHERE ID_Cliente = 6850985'
pd.read_sql(sql = text(query), con = engine.connect()) #--- No puedo usar este para DELETE porque no retorna filas

# al insertar me quedo mal un texto , lo ajuste en el query ejecute de nuevo y quede con 2 registro con el mismo ID_cliente, por eso procedo a borrar los dos registros e insertarlos de nuevo
#query = 'DELETE FROM clientes WHERE ID_Cliente = 6850985'
#engine.connect().execute(text(query))

,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda
0,6850985,33,Doctorado,Soltero,1,Empleado,TI,2,290000.0,0,Casa/Departamento


In [45]:
# consultar toda la tabla
pd.read_sql_table('clientes', con = engine.connect())

,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda
0,5008804,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
1,5008805,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
2,5008806,58,Nivel intermedio,Casado,2,Empleado,Seguridad,3,112500.0,1,Casa/Departamento propio
3,5008808,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio
4,5008810,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio
...,...,...,...,...,...,...,...,...,...,...,...
438458,6840222,43,Nivel intermedio,Soltero,1,Empleado,Construcción Civil,8,103500.0,0,Casa/Departamento propio
438459,6841878,22,Nivel superior,Soltero,1,Business Partner,Ventas,1,54000.0,0,Vive con los padres
438460,6842765,59,Nivel intermedio,Casado,2,Pensionista,Otro,0,72000.0,0,Casa/Departamento propio
438461,6842885,51,Nivel intermedio,Casado,2,Empleado,Ventas,3,121500.0,0,Casa/Departamento propio
